In [14]:
from keras.utils import img_to_array,load_img
import numpy as np
import glob
import os
import cv2

In [15]:
video_source_path = './training_videos'
fps = 5
image_store = []

In [16]:
def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

In [17]:
def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

In [19]:
def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)
	#Resize the Image to (227,227,3) for the network to be able to process it.
	image = cv2.resize(img, (227, 227), interpolation = cv2.INTER_AREA) 
	#Convert the Image to Grayscale
	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]
	image_store.append(gray)

In [20]:
#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found",len(videos),"training video(s)" )

Found 18 training video(s)


In [21]:
#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

In [22]:
#Remove old images
remove_old_images(video_source_path+'/frames')

In [23]:
framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)

In [24]:
image_store=np.array(image_store)
a,b,c=image_store.shape
#Reshape to (227,227,batch_size)
image_store.resize(b,c,a)
#Normalize
image_store=(image_store-image_store.mean())/(image_store.std())
#Clip negative Values
image_store=np.clip(image_store,0,1)

In [25]:
np.save('training.npy', image_store)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))

1